In [2]:
import pandas as pd
from zipfile import ZipFile
from  pathlib import Path
import calendar
import datetime
import pickle
#import py7zr

In [3]:
list_of_category = ['Hair Conditioners',
 'Laundry Detergents',
 'MALE B&R',
 'Shampoos',
 'Feminine Care',
 'Diapers']

In [4]:
def get_category(name_of_file):
    cat_dict = {'br': 'MALE B&R',
 'deterg': 'Laundry Detergents',
 'diapers': 'Diapers',
 'femcare': 'Feminine Care',
 'haircond': 'Hair Conditioners',
 'shampoo': 'Shampoos',
 'MALE B&R': 'MALE B&R',
 'Laundry Detergents': 'Laundry Detergents',
 'Diapers': 'Diapers',
 'Feminine Care': 'Feminine Care',
 'Hair Conditioners': 'Hair Conditioners',
 'Shampoos': 'Shampoos'}
    for k in cat_dict.keys():
        if name_of_file.count(k):
            return cat_dict[k]
    else:
        raise ValueError('В названии файла нет категории', name_of_file)   

In [5]:
paths = Path('data/new_format_data')

zip_files = list(paths.glob('*.zip')) 
tmp_zip = Path('./data/tmp/zip').mkdir(parents=True, exist_ok=True)

z_files = list(paths.glob('*.7z'))
tmp_7z = Path('./data/tmp/7z').mkdir(parents=True, exist_ok=True)


for zip_f in zip_files:
    with ZipFile(zip_f) as zip_file: 
        zip_file.extractall(tmp_zip)

for z in z_files:
    with py7zr.SevenZipFile(z, 'r') as archive:
        archive.exctractall(tmp_7z)


for cat in list_of_category:
            df_check = pd.DataFrame() 

NameError: name 'py7zr' is not defined

In [6]:
files = list(Path('./data/tmp/in').rglob('*.parquet') )
categories = list([get_category(name.name) for name in files])
file_df = pd.DataFrame(files,categories).reset_index()
file_df.columns = ['category', 'file']

In [7]:
files[0].parts[-2:]

('demo', 'Diapers_axsm_pg_diapers_product_2021-10-01_12.parquet')

In [7]:
from tqdm import tqdm


In [ ]:
for c in  file_df['category'].unique():
    print(c)
    names = file_df[file_df['category']==c]['file'].values
    categry_df = pd.DataFrame()
    progress_bar = tqdm(names)
    for f in progress_bar:
        progress_bar.set_postfix({'name_of_file': f})
        tmp_df = pd.read_parquet(f) 
        tmp_df['file'] = '/'.join(f.parts[-2:])
        categry_df = pd.concat([categry_df,tmp_df ], ignore_index= True )
        
    categry_df.to_parquet(f'data/tmp/{c}.pq', index=False)

    



In [8]:
pampers = pd.read_parquet(r"C:\Users\nyzhur\git\DT-PG-NZ\data\tmp\Diapers.pq")

In [9]:
legends = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'legends')

In [14]:
clear_pampers = pd.read_parquet(r"C:\Users\nyzhur\git\DT-PG-NZ\data\tmp\промежуточное\Diapers.pq")

In [10]:
features_group = pd.read_excel(r'.\codes\feature_groups.xlsx')
demo_order = pd.read_excel(r'.\codes\demo_order.xlsx')
demo_order['buyers_gr_label'] = demo_order['buyers_gr_label'].astype(str).str.lower()
periods = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='period_lbl')
time_period_type = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='time_period_type')
year = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='year')
segments_order = pd.read_excel(r'.\codes\segments_order.xlsx')
example = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'example')
legends = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'legends')
shop_order = pd.read_excel(r'.\codes\shop_order.xlsx')
today = datetime.datetime.now().strftime('%Y-%m-%d')

#get_low!
list_of_dict = [features_group,demo_order, periods,time_period_type,year,\
                 segments_order,example,legends,shop_order]
for d in list_of_dict:
    d.columns = [c.lower() for c in d.columns ]

fin_cols = example.columns.to_list() + ['shop_code', 'shop_lvls','channel_code', 'file']
columns_dict = legends[['mask','Colum'.lower()]].dropna(subset='mask').set_index('mask').to_dict()['Colum'.lower()]

In [12]:
def get_period_lbl(time_period_type, year, month):
    return f"{time_period_type.split('/')[0]}: {calendar.month_abbr[month]} {year}"

def get_label_num(time_period_type, year, month):
    return f"{time_period_type.split('/')[0]}: {year} ({str(month).zfill(2)}) {calendar.month_abbr[month]}"



def add_period_lbls(df_in, periods,time_period_type):


    time_period_type_dict = {24 :'2MATs/ 104 we', 12:'MAT/ 52 we', 3:'3MMT/ 12we', 1: 'Month'}
    
    col_to_int = ['duration', 'year', 'month']
    df_in[col_to_int] = df_in[col_to_int].astype(int)
    
    df_in['time_period_type'] = df_in['duration'].map(time_period_type_dict)
    
    df_tmp_per = df_in[['time_period_type', 'year', 'month']].drop_duplicates()
    df_tmp_per['period_lbl'] = df_tmp_per[['time_period_type', 'year', 'month']]\
        .apply(lambda row: get_period_lbl(*row), axis=1)
    
    df_tmp_per['label_num'] = df_tmp_per[['time_period_type', 'year', 'month']]\
        .apply(lambda row: get_label_num(*row), axis=1)
    


    new_periods = set(df_tmp_per['period_lbl']) - set(periods['period_lbl'])
    if len(new_periods):
        print('New periods!')
        print(*new_periods)
        new_periods_df = df_tmp_per[df_tmp_per['period_lbl'].isin(new_periods)].sort_values(by=['year','month'])
        new_periods_df['date_added'] = today
        periods = pd.concat([periods[['period_lbl','label_num','date_added' ]], new_periods_df[['period_lbl','label_num','date_added' ]]],ignore_index=True )
        periods['period_code'] = range(1,len(periods)+1)


        with pd.ExcelWriter(r'.\codes\periods.xlsx',engine="openpyxl",mode="a", if_sheet_exists='replace') as writer:
            periods.to_excel(writer, sheet_name ='period_lbl')





    df_in = df_in.merge(df_tmp_per,on = ['time_period_type', 'year', 'month'], how='left')
    
    df_in = df_in.merge(
            periods, on=['period_lbl', 'label_num'],  how='left', validate='many_to_one')
    
    
    
    df_in = df_in.merge(
           time_period_type, on='time_period_type', how='left', validate='many_to_one')

    df_in = df_in.drop(columns=['time_period_type', 'period_lbl', 'month' ])\
            .rename(columns={'time_period_code': 'time_period_type', 'period_code': 'period_lbl'})

    return df_in

def get_df_in_v2(df_in, category, columns_dict, features_group):
    
    #get_low_2!
    df_in.columns = [c.lower() for c in df_in.columns]


    df_in = df_in.rename(
        {'Category Name'.lower():'Product Name'.lower(), 'Buyer Group Name'.lower():'buyers_gr_label'}, 
        axis=1)
    df_in['Category Name'.lower()] = category
    df_in['buyers_gr_label'] = df_in['buyers_gr_label'].replace({'Total Population' :'TOTAL DEMOGRAPHICS'})
    df_in = df_in.drop('category',axis=1)


    #print('init shape', df_in.shape)


        
    # products
    prod_col_to_merge = 'Product Name'.lower()
    if  len(set(df_in['Product Name'.lower()].unique()) - set(features_group['Product Name'.lower()]))>0:
        print('Мерджим по фулл нейму')
        prod_col_to_merge = 'full_label'
        features_group = features_group.drop(columns='Product Name'.lower())

    
    df_in = df_in.merge(
        features_group, 
        left_on=['Category Name'.lower(), 'Product Name'.lower()], 
        right_on=['Category Name'.lower(),  prod_col_to_merge], 
        how='left', validate='many_to_one') #Есть множественные ключи в features_group['Product Name']
    




    df_in = df_in.rename(
        columns={'product_code': 'products', 'category_code': 'category', 'feature_code': 'features'}
    )
    df_in['product_hier'] = df_in['products']


    # socdem
    df_in['buyers_gr_label'] = df_in['buyers_gr_label'].str.lower()
    df_in = df_in.merge(
        demo_order, on=['buyers_gr_label'], how='left', validate='many_to_one')
    df_in = df_in.rename(columns={'demo_code': 'demo_groups'})
    df_in['demo_hier'] = df_in['demo_groups']

    
    # segments

    if ('Segment'.lower() not in df_in.columns):
        if (category == 'Feminine Care'):
            df_in['Segment'.lower()] = 'Total Feminine Care'
        elif (category == 'Shampoos'):
            df_in['Segment'.lower()] = 'Shampoos'
    

    
    if (category == 'Laundry Detergents'):
        df_in['Segment'.lower()] = df_in['Segment'.lower()].replace(
            {'Total Detergents excluding Bars': 'Total Detergents excluding Bar'})
    df_in['Segment'.lower()] =df_in['Segment'.lower()].replace({'Total Diapers size':'Total Diapers'})
    df_in = df_in.merge(
        segments_order, on=['Segment'.lower()], how='left', validate='many_to_one')
    df_in = df_in.rename(columns={'segment_code': 'cat_segment'})


    # shop
    df_in['position_name_shop'] = df_in['position_name_shop'].replace(
    {'Ok Hyper': 'OK Hyper', 'Perekryostok': 'Perekrestok'})
    df_in = df_in.merge(
        shop_order, on=['position_name_shop'], how='left', validate='many_to_one')

    
    #print('merged shape', df_in.shape)
    
    # rename metrics
    df_in.columns = [col.lower().replace(' ','_') for col in df_in.columns]
    df_in = df_in.rename(columns_dict, axis=1)

    
    return df_in

In [15]:
set(clear_pampers['Category Name'.lower()].unique()) - set(features_group['Product Name'.lower()])

{'Hellen Harper Soft&Dry Non-Taped',
 'Pampers T2 Non-Taped Total Total',
 'Retailer brand 365 Dney Non-Taped',
 'Retailer brand Baby Go Non-Taped'}

In [17]:
set(clear_pampers['Category Name'.lower()].unique()) - set(features_group['full_label'.lower()])

set()

In [18]:
res = get_df_in_v2(clear_pampers, 'Diapers', columns_dict, features_group)

Мерджим по фулл нейму


In [19]:
res.head()

,product_name,segment,buyers_gr_label,position_name_shop,spend_local_currency,volume_gl,volume_physical_units,volume_su,trips_raw,occasions,...,demo_lvls,socdem_gr,cat_segment,shop_code,hier,shop_lvls,shop_hier,channel_type,channel_code,shop_name
0,Diapers,Total Diapers,total demographics,RUSSIA NATIONAL,4.812561e+07,2624.097997,14578.438644,57000.820444,12878.0,43715.265072,...,0.0,1.0,100,1,1.0.0,1,1.0.0 RUSSIA NATIONAL,Total,1,RUSSIA NATIONAL
1,Diapers,Total Diapers,size of households,RUSSIA NATIONAL,4.812561e+07,2624.097997,14578.438644,57000.820444,12878.0,43715.265072,...,1.0,8.0,100,1,1.0.0,1,1.0.0 RUSSIA NATIONAL,Total,1,RUSSIA NATIONAL
2,Diapers,Total Diapers,1,RUSSIA NATIONAL,1.828382e+06,96.941043,538.565648,2425.211514,571.0,1984.690582,...,2.0,8.0,100,1,1.0.0,1,1.0.0 RUSSIA NATIONAL,Total,1,RUSSIA NATIONAL
3,Diapers,Total Diapers,2,RUSSIA NATIONAL,4.277845e+06,239.037002,1327.993951,5096.243527,1024.0,3783.258468,...,2.0,8.0,100,1,1.0.0,1,1.0.0 RUSSIA NATIONAL,Total,1,RUSSIA NATIONAL
4,Diapers,Total Diapers,3,RUSSIA NATIONAL,1.424264e+07,734.939132,4083.027791,14500.245537,3449.0,11058.175486,...,2.0,8.0,100,1,1.0.0,1,1.0.0 RUSSIA NATIONAL,Total,1,RUSSIA NATIONAL


In [20]:
chk_na = res.isna().sum()    
if chk_na[chk_na > 0].shape[0] > 0:
        print('WARNING\n', 'columns with na:')
        print(chk_na[chk_na > 0]) 

WARNING
 columns with na:
buyer_group_id    114604
demo_groups       114604
demo_hier         114604
demo_lvls         114604
socdem_gr         114604
dtype: int64


In [94]:
res[res['buyer_group_id'].isna()]['buyers_gr_label'].unique()

array(['young singles/couples without children (<35 years)',
       'mature singles/couples without children (35-49 years)',
       'senior singles/couples without children (50-64 years)',
       'retired singles/couples without children (>65 years)',
       'family with children 6-12 years',
       'family with children 13-18 years'], dtype=object)

In [23]:
res['file'].tail()

1842068    shops/Diapers_axsm_pg_diapers_product_2024-09-...
1842069    shops/Diapers_axsm_pg_diapers_product_2024-09-...
1842070    shops/Diapers_axsm_pg_diapers_product_2024-09-...
1842071    shops/Diapers_axsm_pg_diapers_product_2024-09-...
1842072    shops/Diapers_axsm_pg_diapers_product_2024-09-...
Name: file, dtype: object